In [1]:
import os
%pwd

'd:\\Text Sumarizer\\researrch'

In [2]:
os.chdir("../")

In [10]:
## update config yaml
## updating parameters in params yaml
## Config Manager
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    ## Config yaml
    root_dir: Path
    data_path: Path
    model_ckpt: str
    ## params yaml
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int


In [11]:
from src.text_summarizer.constants import *
from src.text_summarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_PATH, params_filepath=PARAMS_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create necessary directories
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:  # <-- Moved outside __init__
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])  # Ensure directory exists

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            per_device_eval_batch_size=params.per_device_eval_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,    
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )

        return model_trainer_config


In [12]:
## Model Trainer Component
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
import torch
from datasets import load_dataset, load_from_disk

In [13]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device='cuda' if torch.cuda.is_available() else 'cpu'
        ## Loading tokenizer
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        ## Model
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)
        sequential_dataset = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus) 

        ##loading dataset
        dataset_samsum = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,    
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )
        trainer=Trainer(model=model_pegasus, args=trainer_args,
                tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                train_dataset=dataset_samsum_pt['test'],
                eval_dataset=dataset_samsum_pt['validation'])

        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))
        

In [14]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

^C


In [ ]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()   

[2025-03-13 23:24:21,527: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-03-13 23:24:21,532: INFO: common]: yaml file: params.yaml loaded successfully
[2025-03-13 23:24:21,535: INFO: common]: created directory at: Artifacts/data_ingestion
[2025-03-13 23:24:21,538: INFO: common]: created directory at: Artifacts/model_trainer


d:\Text Sumarizer\env\Lib\site-packages\huggingface_hub\file_download.py:653: UserWarning: Not enough free disk space to download the file. The expected file size is: 2275.26 MB. The target location C:\Users\Micro Tech\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail\blobs only has 293.59 MB free disk space.
  warnings.warn(
